In [1]:
#libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import wrangle #acquire, prep and clean library
import env
import explore

from scipy import stats
from scipy.stats import pearsonr, spearmanr

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, QuantileTransformer
from sklearn.model_selection import train_test_split

from math import sqrt

# Notes

# Planning

# Data Acquisition

In [2]:
df = wrangle.get_zillow_data()
df.head(2)

,parcelid,bathroomcnt,bedroomcnt,fips,garagecarcnt,lotsizesquarefeet,yearbuilt,taxvaluedollarcnt,taxamount,calculatedfinishedsquarefeet,predictions
0,17173851,2.0,4.0,6111.0,NaN,11396.0,1952.0,64712.0,781.00,1848.0,0.088469
1,10821119,2.0,2.0,6037.0,NaN,6299.0,1952.0,381638.0,4692.27,1430.0,0.025629


In [3]:
df.shape

(2152985, 11)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152985 entries, 0 to 2152984
Data columns (total 11 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   parcelid                      int64  
 1   bathroomcnt                   float64
 2   bedroomcnt                    float64
 3   fips                          float64
 4   garagecarcnt                  float64
 5   lotsizesquarefeet             float64
 6   yearbuilt                     float64
 7   taxvaluedollarcnt             float64
 8   taxamount                     float64
 9   calculatedfinishedsquarefeet  float64
 10  predictions                   float64
dtypes: float64(10), int64(1)
memory usage: 180.7 MB


In [5]:
df.isnull().sum()

parcelid                              0
bathroomcnt                          11
bedroomcnt                           11
fips                                  0
garagecarcnt                    1454321
lotsizesquarefeet                 13441
yearbuilt                          9339
taxvaluedollarcnt                   493
taxamount                          4442
calculatedfinishedsquarefeet       8485
predictions                     2100543
dtype: int64

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,2152985.0,1.309662e+07,5.172434e+06,1.071172e+07,1.165907e+07,1.259850e+07,1.405608e+07,1.696019e+08
bathroomcnt,2152974.0,2.230677e+00,9.992790e-01,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,3.200000e+01
bedroomcnt,2152974.0,3.287195e+00,9.547910e-01,0.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,2.500000e+01
fips,2152985.0,6.048377e+03,2.043332e+01,6.037000e+03,6.037000e+03,6.037000e+03,6.059000e+03,6.111000e+03
garagecarcnt,698664.0,1.926653e+00,5.696367e-01,0.000000e+00,2.000000e+00,2.000000e+00,2.000000e+00,2.500000e+01
lotsizesquarefeet,2139544.0,1.107215e+04,2.656322e+05,1.000000e+02,5.633000e+03,6.760000e+03,8.551000e+03,3.710005e+08
yearbuilt,2143646.0,1.960949e+03,2.216238e+01,1.801000e+03,1.949000e+03,1.958000e+03,1.976000e+03,2.016000e+03
taxvaluedollarcnt,2152492.0,4.618967e+05,6.996805e+05,1.000000e+00,1.881638e+05,3.276710e+05,5.345212e+05,9.842891e+07
taxamount,2148543.0,5.634881e+03,8.179022e+03,1.850000e+00,2.534920e+03,4.108900e+03,6.414300e+03,1.337756e+06
calculatedfinishedsquarefeet,2144500.0,1.862852e+03,1.222120e+03,1.000000e+00,1.257000e+03,1.623000e+03,2.208000e+03,9.525760e+05


In [8]:
df = df.drop_duplicates()

In [9]:
df.shape

(2152982, 11)

In [10]:
df = df.rename(columns={"bedroomcnt": "bedrooms", "bathroomcnt": "bathrooms", "calculatedfinishedsquarefeet":"square_feet", "taxvaluedollarcnt":"tax_value", "yearbuilt":"year_built", "taxamount":"tax_amount", "fips":"county_code", "garagecarcnt":"garage", "lotsizesquarefeet":"lot_size"})
df.head(0)

,parcelid,bathrooms,bedrooms,county_code,garage,lot_size,year_built,tax_value,tax_amount,square_feet,predictions


In [37]:

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152982 entries, 0 to 2152984
Data columns (total 11 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   parcelid     object
 1   bathrooms    int64 
 2   bedrooms     int64 
 3   county_code  object
 4   garage       int64 
 5   lot_size     object
 6   year_built   object
 7   tax_value    int64 
 8   tax_amount   int64 
 9   square_feet  int64 
 10  predictions  int64 
dtypes: int64(7), object(4)
memory usage: 197.1+ MB


In [17]:
df['bedrooms'] = df.bedrooms.fillna(value = 3)

In [18]:
df['bathrooms'] = df.bathrooms.fillna(value = 2.5)

In [21]:
df['garage'] = df.garage.fillna(value = 2)

In [23]:
df['tax_amount'] = df.tax_amount.fillna(value = df['tax_amount'].mean())

In [25]:
df['tax_value'] = df.tax_value.fillna(value = 45000)

In [27]:
df['square_feet'] = df.square_feet.fillna(value = df['square_feet'].mean())

In [33]:
df['predictions'] = df.predictions.fillna(value = df['predictions'].mean())

In [40]:
df['lot_size'] = df.lot_size.fillna(value = df['lot_size'].mean())

In [41]:
#converts floats to int 
convert_dict_int = {'bathrooms': int, 'bedrooms': int, 
                   'garage':int, 'tax_amount':int, 'tax_value':int,
                   'square_feet':int, 'predictions':int, 'lot_size':int}
df = df.astype(convert_dict_int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2152982 entries, 0 to 2152984
Data columns (total 11 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   parcelid     object
 1   bathrooms    int64 
 2   bedrooms     int64 
 3   county_code  object
 4   garage       int64 
 5   lot_size     int64 
 6   year_built   object
 7   tax_value    int64 
 8   tax_amount   int64 
 9   square_feet  int64 
 10  predictions  int64 
dtypes: int64(8), object(3)
memory usage: 197.1+ MB


##### NOTES

1. SQL zillow dataset
2. property type - Single Family - 261
3. .shape (2152863, 61)
4. After initial data acquisition .shape (2152863, 16)
    - From initial acquisition I kept the following columns:
    - parcelid, basementsqft, bathroomcnt, bedroomcnt, fips, garagecarcnt, garagetotalsqft, lotsizesquarefeet, poolcnt, poolsizesum, roomcnt, unitcnt, yearbuilt, numberofstories, taxvaluedollarcnt, taxamount
5. Dropped (based on qty of nulls): basementsqft, garagetotalsqft, poolcnt, poolsizesum, unitcnt, numberofstories, roomcnt
6. Final .shape (2152985, 11)
7. Drop duplicates (2152982, 11) 
8. Fill in nulls (as identified in wrangle.py)
9. Drop columns
10. Visualize
11. Remove outliers

# Data Preparation

# Data Exploration

### Univariate

In [ ]:
explore.explore_univariate(train, cat_vars, quant_vars)

### Bivariate

In [ ]:
#exploring split
explore.explore_bivariate(train, target, cat_vars, quant_vars)

### Multivariate

In [ ]:
#exploring  split
explore.explore_multivariate(train, target, cat_vars, quant_vars)

# Hypothesis Testing

# Modeling